In [1]:
import os
import ast
import networkx as nx
from copy import deepcopy

In [2]:
### open Hamiltonian data ###

working_dir = os.getcwd()
parent_dir = os.path.dirname(working_dir) # gets directory where running python file is!

data_dir = os.path.join(parent_dir, 'Molecular_Hamiltonian_data')
hamiltonian_data = os.path.join(data_dir, 'hamiltonians.txt')

In [3]:
with open(hamiltonian_data, 'r') as input_file:
    hamiltonians = ast.literal_eval(input_file.read())

for key in hamiltonians.keys():
    print(f"{key: <25}     n_qubits:  {hamiltonians[key][1]:<5.0f}")

H2-S1_STO-3G_singlet          n_qubits:  18   
C1-O1_STO-3G_singlet          n_qubits:  16   
H1-Cl1_STO-3G_singlet         n_qubits:  16   
H1-Na1_STO-3G_singlet         n_qubits:  16   
H2-Mg1_STO-3G_singlet         n_qubits:  17   
H1-F1_3-21G_singlet           n_qubits:  18   
H1-Li1_3-21G_singlet          n_qubits:  18   
Be1_STO-3G_singlet            n_qubits:  5    
H1-F1_STO-3G_singlet          n_qubits:  8    
H1-Li1_STO-3G_singlet         n_qubits:  8    
Ar1_STO-3G_singlet            n_qubits:  13   
F2_STO-3G_singlet             n_qubits:  15   
H1-O1_STO-3G_singlet          n_qubits:  8    
H2-Be1_STO-3G_singlet         n_qubits:  9    
H2-O1_STO-3G_singlet          n_qubits:  10   
H2_3-21G_singlet              n_qubits:  5    
H2_6-31G_singlet              n_qubits:  5    
H3-N1_STO-3G_singlet          n_qubits:  13   
H4-C1_STO-3G_singlet          n_qubits:  14   
Mg1_STO-3G_singlet            n_qubits:  13   
N2_STO-3G_singlet             n_qubits:  15   
Ne1_STO-3G_si

In [4]:
molecule_key = 'H1-Li1_STO-3G_singlet'
transformation, N_qubits, Hamilt_dictionary, _ ,_, _ = hamiltonians['H1-Li1_STO-3G_singlet']

# 1. Get OpenFermion representation of Hamiltonian

In [5]:
from quchem.Misc_functions.conversion_scripts import Get_Openfermion_Hamiltonian

openFermion_H = Get_Openfermion_Hamiltonian(Hamilt_dictionary)
openFermion_H

-5.001425458221718 [] +
0.004877026695939432 [X0] +
0.008434569756845523 [X0 X1] +
0.003463638100879079 [X0 Z1 X2 Z3 Z5 Z7] +
-0.0034636381008790766 [X0 Z1 Z2 X3 Z5 Z7] +
0.004882949606014626 [X0 Z1 Z2 X4 Z5 Z7] +
-0.004882949606014626 [X0 Z1 Z2 Z4 X5 Z7] +
-0.0015024523676698824 [X0 Z1 Z2 Z4 X6 Z7] +
-0.004877026695939432 [X0 Z1 Z2 Z4 Z6] +
0.0015024523676698824 [X0 Z1 Z2 Z4 Z6 X7] +
-0.004877026695939432 [X0 Z1 Z3 Z5 Z7] +
-0.003463638100879079 [X0 X2] +
0.010327309992822217 [X0 X2 X3] +
0.004803863260481915 [X0 X2 Z3 Z4 X5] +
-0.0025611654019874314 [X0 X2 Z3 Z4 Z5 Z6 X7] +
-0.010327309992822217 [X0 Y2 Y3] +
-0.004803863260481915 [X0 Y2 Z3 Z4 Y5] +
0.0025611654019874314 [X0 Y2 Z3 Z4 Z5 Z6 Y7] +
-0.004882949606014626 [X0 Z2 Z3 X4] +
0.0015024523676698824 [X0 Z2 Z3 Z4 Z5 X6] +
0.004877026695939432 [X0 Z2 Z3 Z4 Z5 Z6 Z7] +
-0.0048038632604819175 [X0 X3 X4] +
0.0025611654019874314 [X0 X3 Z4 Z5 X6] +
0.0034636381008790766 [X0 X3 Z4 Z5 Z6 Z7] +
0.0048038632604819175 [X0 Y3 Y4] +
-0.0025611

# 2. Get cliques defined by commutativity 


In [6]:
from quchem.Unitary_Partitioning.Graph import Vector_QubitHamiltonian

In [7]:
n_qubits = 8
clique_relation = 'C'

Hamilt_vector = Vector_QubitHamiltonian(openFermion_H, n_qubits)

adj_matrix = Hamilt_vector.Get_adj_mat(clique_relation)
node_conv_dict = {ind: Pvec.QubitOp for ind, Pvec in Hamilt_vector.Qubit_H_vec_ind_to_Pvec.items()}

Graph = nx.from_numpy_matrix(adj_matrix)
Complement_Graph = nx.complement(Graph)

In [10]:
max_cliques = nx.algorithms.clique.find_cliques(Complement_Graph)

In [11]:
len(max_cliques)

TypeError: object of type 'generator' has no len()

In [24]:
list_cliques = []
for  
first_maximal = next(max_cliques)

In [27]:
xxx = nx.algorithms.approximation.clique.max_clique(Complement_Graph)

AttributeError: module 'networkx.algorithms' has no attribute 'approximation'

In [25]:
first_maximal

[1, 512, 132, 419, 125, 64, 220, 12]

In [66]:
from quchem.Unitary_Partitioning.Graph import Clique_cover_Hamiltonian

commutativity_flag = 'C' ## <- defines relationship between sets!!!
Graph_colouring_strategy='largest_first'


Commuting_sets = Clique_cover_Hamiltonian(openFermion_H, 
                                                     N_qubits, 
                                                     commutativity_flag, 
                                                     Graph_colouring_strategy)
Commuting_sets

{0: [-5.001425458221718 []],
 1: [-0.4579624763565425 [Z1],
  0.008434569756845523 [X0 X1],
  -0.0015024523676698824 [Y1 Y7],
  0.0024547625653344985 [Y1 Y6 X7],
  -0.0018915758280655941 [Y1 Y4 Z5 Z6 X7],
  -0.0025611654019874314 [Y1 Y2 Z3 Z4 Z5 Z6 X7]],
 2: [-0.4579624763565428 [Z0],
  0.008434569756845523 [Y0 Y1],
  0.004877026695939432 [X0],
  0.003463638100879079 [Y0 Z1 Y2 Z3 Z5 Z7],
  0.004882949606014626 [Y0 Z1 Z2 Y4 Z5 Z7],
  -0.0015024523676698824 [Y0 Z1 Z2 Z4 Y6 Z7]],
 3: [0.2707726623751819 [Z0 Z1],
  0.004877026695939432 [X1],
  0.0024547625653344985 [Y1 X6 Y7],
  0.0018915758280655941 [Y1 X5 Y6],
  0.0025611654019874314 [Y1 X3 Z4 Z5 Y6],
  0.0015024523676698824 [Y1 Z2 Z3 Z4 Z5 Y6],
  0.003463638100879079 [Z0 Y1 Y2 Z3 Z5 Z7],
  0.004882949606014626 [Z0 Y1 Z2 Y4 Z5 Z7]],
 4: [1.0104418175624195 [Z7],
  -0.015075372035589736 [X5 Z6 X7],
  -0.025703421671713647 [X3 Z4 Z5 Z6 X7],
  -0.0002823107691970594 [Z2 Z3 Z4 Z5 Z6 X7],
  0.0024547625653344985 [X1 X6 X7],
  -0.0018915758280

In [67]:
key_larg, largest_C_set = max(Commuting_sets.items(), key=lambda x:len(x[1])) 

In [68]:
sorted_cliques = sorted(list(Commuting_sets.values()), key=lambda x:len(x))

In [69]:
sorted_cliques[-1] == largest_C_set

True

In [70]:
largest_C_set

[1.0104418175624195 [Z7],
 -0.015075372035589736 [X5 Z6 X7],
 -0.025703421671713647 [X3 Z4 Z5 Z6 X7],
 -0.0002823107691970594 [Z2 Z3 Z4 Z5 Z6 X7],
 0.0024547625653344985 [X1 X6 X7],
 -0.0018915758280655941 [X1 X4 Z5 Z6 X7],
 -0.0025611654019874314 [X1 X2 Z3 Z4 Z5 Z6 X7],
 -0.0015024523676698824 [Y0 Y7],
 0.0024547625653344985 [Y0 Y6 X7],
 -0.0018915758280655941 [Y0 Y4 Z5 Z6 X7],
 -0.0025611654019874314 [Y0 Y2 Z3 Z4 Z5 Z6 X7],
 0.0015024523676698824 [Z0 Y1 Z2 Z4 Z6 Y7]]

In [17]:
import sys
sys.path.append( '../Projects/CS_VQE/' )

In [18]:
import cs_vqe as c

In [19]:
nonH_guesses = c.greedy_dfs(Hamilt_dictionary, 10, criterion='weight')

nonH = max(nonH_guesses, key=lambda x:len(x)) # largest nonCon part found by dfs alg
len(nonH)

74

In [20]:
nonCon_H = {}
for P in nonH:
    if P in Hamilt_dictionary.keys():
        nonCon_H[P]=Hamilt_dictionary[P]
c.contextualQ_ham(nonCon_H)    

False

In [31]:
nonCon_H

In [71]:
from quchem.Misc_functions.conversion_scripts import Openfermion_to_dict
from functools import reduce

Q_op_commuting = reduce(lambda x,y: x+y, largest_C_set)
N_qubits = len(list(nonCon_H.keys())[0])

commuting_dict = Openfermion_to_dict(Q_op_commuting, N_qubits)

In [29]:
len(commuting_dict)

34

In [72]:
H_full = Hamilt_dictionary

H_remaining = {}
for term in Hamilt_dictionary:
    if term in commuting_dict:
        continue
    else:
        H_remaining[term]=Hamilt_dictionary[term]

In [73]:
c.contextualQ_ham(commuting_dict)

False

In [74]:
c.contextualQ(H_remaining)

KeyError: 0

In [75]:
from datetime import datetime
from datetime import timedelta

cutoff = 10 # seconds

start_time = datetime.now()
delta = timedelta(seconds=cutoff)

H_noncon_Graph = deepcopy(commuting_dict)

key_list_by_weight = [k for k, v in sorted(H_remaining.items(), key=lambda item: -item[1])]
while (datetime.now()-start_time) and len(key_list_by_weight)>0:
    
    
    active_key = key_list_by_weight.pop()
    H_noncon_Graph[active_key] = H_remaining[active_key]
    if c.contextualQ_ham(H_noncon_Graph):
        del H_noncon_Graph[active_key]

In [76]:
H_noncon_Graph

{'IIIIIIIZ': 1.0104418175624195,
 'IIIIIXZX': -0.015075372035589736,
 'IIIXZZZX': -0.025703421671713647,
 'IIZZZZZX': -0.0002823107691970594,
 'IXIIIIXX': 0.0024547625653344985,
 'IXIIXZZX': -0.0018915758280655941,
 'IXXZZZZX': -0.0025611654019874314,
 'YIIIIIIY': -0.0015024523676698824,
 'YIIIIIYX': 0.0024547625653344985,
 'YIIIYZZX': -0.0018915758280655941,
 'YIYZZZZX': -0.0025611654019874314,
 'ZYZIZIZY': 0.0015024523676698824,
 'IIIIIIII': -5.001425458221718}

In [76]:
def greedy_dfs(ham,cutoff,criterion='weight'):
    
    weight = {k:abs(ham[k]) for k in ham.keys()}
    possibilities = [k for k, v in sorted(weight.items(), key=lambda item: -item[1])] # sort in decreasing order of weight
    
    best_guesses = [[]]
    stack = [[[],0]]

    
    i = 0
    
    while datetime.now()-start_time < delta and stack:
        
        while i < len(possibilities):
#             print(i)
            next_set = stack[-1][0]+[possibilities[i]]
#             print(next_set)
#             iscontextual = contextualQ(next_set)
#             print('  ',iscontextual,'\n')
            if not contextualQ(next_set):
                stack.append([next_set,i+1])
            i += 1
        
        if criterion == 'weight':
            new_weight = sum([abs(ham[p]) for p in stack[-1][0]])
            old_weight = sum([abs(ham[p]) for p in best_guesses[-1]])
            if new_weight > old_weight:
                best_guesses.append(stack[-1][0])
                # print(len(stack[-1][0]))
                # print(stack[-1][0],'\n')
            
        if criterion == 'size' and len(stack[-1][0]) > len(best_guesses[-1]):
            best_guesses.append(stack[-1][0])
            # print(len(stack[-1][0]))
            # print(stack[-1][0],'\n')
            
        top = stack.pop()
        i = top[1]
    
    return best_guesses

In [128]:
commuting_H = reduce(lambda x,y: x+y, largest_C_set)

In [129]:
commuting_H_VEC = Vector_QubitHamiltonian(commuting_H, n_qubits)

for key in Commuting_sets:
    if key != key_larg:
        for P_op in Commuting_sets[key]:
            

SyntaxError: unexpected EOF while parsing (<ipython-input-129-edd426d09be6>, line 6)

In [94]:
comm_H_vecs = [VectorPauliWord(8, op) for op in commuting_H]

In [89]:
all([True, False])

False

In [103]:
# from quchem.Unitary_Partitioning.Graph import Commutes, VectorPauliWord

# allowed_terms =[]

# for key in Commuting_sets:
#     if key != key_larg:
#         for P_op in Commuting_sets[key]:
#             vec_op = VectorPauliWord(8, P_op)
#             x = all([Commutes(vec_op, op) for op in comm_H_vecs])
#             print(x)
#             if x:
#                 allowed_terms.append(P_op)
            

In [109]:
import quchem.Misc_functions.conversion_scripts as conv_scr
comm_H_dict = conv_scr.Openfermion_to_dict(commuting_H,  8) 

In [133]:
remaining_terms_dict ={}

for op, val in Hamilt_dictionary.items():
    if op in comm_H_dict:
        continue
    else:
        remaining_terms_dict[op] = val
    

In [134]:
remining_by_size = sorted([(op, coeff) for op, coeff in remaining_terms_dict.items()], key=lambda x: x[1])

In [146]:
for op, coeff in remining_by_size:
    comm_H_dict[op]= coeff
    if not c.contextualQ_ham(comm_H_dict):
        continue
    else:
        del comm_H_dict[op]
        print('BAD')
        break

BAD


In [144]:
for op, coeff in remining_by_size:
    new_set = deepcopy(remaining_terms_dict)
    new_set[op]= coeff
    if c.contextualQ_ham(new_set):
        continue
    else:
        remaining_terms_dict = new_set

In [148]:
len(comm_H_dict)

35

In [150]:
c.contextualQ_ham(nonH)

AttributeError: 'list' object has no attribute 'keys'

False